In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\HAPPYUSER\\Documents\\Web_Development\\ML_AI_MLOPS\\Sports-image-classification'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_random_rotation: float
    params_random_contrast: float
    params_random_translation_width: float
    params_random_translation_height: float

In [12]:
from SportsImageClassifier.constants import *
from SportsImageClassifier.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_random_rotation=self.params.RANDOM_ROTATION_FACTOR,
            params_random_contrast=self.params.RANDOM_CONTRAST_FACTOR,
            params_random_translation_width=self.params.RANDOM_TRANSLATION_WIDTH_FACTOR,
            params_random_translation_height=self.params.RANDOM_TRANSLATION_HEIGHT_FACTOR
        )

        return prepare_base_model_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [15]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
            )

        self.save_model(path=self.config.base_model_path, model=self.model)


    def image_augmentation(self):
        img_augmentation = Sequential(
        [
            layers.RandomRotation(factor=self.config.params_random_rotation),
            layers.RandomTranslation(height_factor=self.config.params_random_translation_height, width_factor=self.config.params_random_translation_width),
            layers.RandomFlip(),
            layers.RandomContrast(factor=self.config.params_random_contrast),
        ],
        name="img_augmentation",
            )
        return img_augmentation


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, img_augmentation):
        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                layer.trainable = False
        
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        prediction_layer = tf.keras.layers.Dense(classes)
        inputs = tf.keras.Input(shape=(256, 256, 3))
        x = img_augmentation(inputs)
        x = preprocess_input(x)
        x = model(x, training=False)
        x = global_average_layer(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = prediction_layer(x)
        full_model = tf.keras.Model(inputs, outputs)
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            img_augmentation=self.image_augmentation()
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [17]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-09-02 20:25:27,110: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 20:25:27,114: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 20:25:27,117: INFO: common: created directory at: artifacts]
[2024-09-02 20:25:27,119: INFO: common: created directory at: artifacts/prepare_base_model]


ValueError: File format not supported: filepath=False. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

'artifacts/prepare_base_model'